In [14]:
import shutil
import os
from PIL import Image

In [49]:
ann_dir = 'annotations/'
file_names = os.listdir(ann_dir)
img_dir = 'train_renamed/'
yolo_ann_dir = 'yolo_annotations/'

In [4]:
labels = []

In [5]:
for file_name in file_names:
    
    label = file_name.split('_')[0]
    labels.append(label)

In [7]:
len(set(labels))

120

In [47]:
def convert_back(size, box):
    
    #bad naming, yeah
    
    img_w = size[0]
    img_h = size[1]
    
    x_b_c = box[0]  #box_converted
    y_b_c = box[1]
    w_b_c = box[2]
    h_b_c = box[3]
    
    x_b = x_b_c * img_w #box
    w_b = w_b_c * img_w
    y_b = y_b_c * img_h
    h_b = h_b_c * img_h
    
    w_half = w_b / 2
    h_half = h_b / 2
    
    x_min = x_b - w_half
    y_min = y_b - h_half
    x_max = x_b + w_half
    y_max = y_b + h_half
    
    
    return (x_min, y_min, x_max, y_max)

In [38]:
def convert(size, box):
    dw = 1 / size[0]
    dh = 1 / size[1]
    x = (box[0] + box[1]) / 2
    y = (box[2] + box[3]) / 2
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

In [56]:
for file_name in file_names:
    file_path = os.path.join(ann_dir, file_name)
    
        
    lines = []
    with open(file_path) as f:
        lines = f.readlines()
        
    label = lines[0].strip()
    elems = lines[1].split(' ')
    
    x_min = int(elems[0])
    x_max = int(elems[2])
    y_min = int(elems[1])
    y_max = int(elems[3])
        
    img_name = file_name.split('.')[0] + '.jpg'
    
    img_path = os.path.join(img_dir, img_name)
        
    img = Image.open(img_path)
    w, h = img.size
    
    box = (x_min, x_max, y_min, y_max)
    converted_box = convert((w, h), box) #[object center in X] [object center in Y] [object width in X] [object width in Y]
    
    converted_back_box = convert_back((w, h), converted_box)
    
    
    with open(os.path.join(yolo_ann_dir, file_name), 'w') as f:
        f.write('{} {}'.format(label, ' '.join([str(item) for item in converted_box])))
